In [17]:
from pyspark.sql import SparkSession

In [18]:

spark = SparkSession.builder \
    .appName('Ingest checkin table into bronze') \
    .master('spark://spark-master:7077') \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083")\
    .config("spark.hadoop.fs.s3a.access.key", 'minio') \
    .config("spark.hadoop.fs.s3a.secret.key", 'minio123') \
    .config("spark.hadoop.fs.s3a.endpoint", 'minio:9000')\
    .config("spark.hadoop.fs.s3a.path.style.access", "true")\
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")\
    .config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')\
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .config('spark.sql.warehouse.dir', f's3a://lakehouse/')\
    .enableHiveSupport()\
    .getOrCreate()

In [19]:
spark.sql("CREATE DATABASE Image")

DataFrame[]

In [20]:
imagesDF = spark.read.format("image").load("s3a://mlflow/2/a860566fa4364a2fac7828e0eb69e4ef/artifacts/elbow_method.png")


In [21]:
imagesDF.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)



In [27]:
imagesDF.show()

+--------------------+
|               image|
+--------------------+
|{s3a://mlflow/2/a...|
+--------------------+



In [28]:
imagesDF.write.format("delta").saveAsTable("Image.elbow")

In [41]:
imagesDF.printSchema()

root
 |-- image: struct (nullable = true)
 |    |-- origin: string (nullable = true)
 |    |-- height: integer (nullable = true)
 |    |-- width: integer (nullable = true)
 |    |-- nChannels: integer (nullable = true)
 |    |-- mode: integer (nullable = true)
 |    |-- data: binary (nullable = true)



In [54]:
spark.sql("""
SELECT *
FROM Image.elbow
""").show()



+--------------------+
|               image|
+--------------------+
|{s3a://mlflow/2/a...|
+--------------------+

